In [ ]:
# **Análisis y Limpieza de un Dataset Económico**
### **Nombre del Alumno: Alejandro de Santiago Vicente**
### **Fecha: 28 de octubre de 2024**

![Imagen relacionada con datos económicos](https://upload.wikimedia.org/wikipedia/commons/6/6a/Data-science-icon.png)

---

Este notebook presenta la resolución del problema de análisis y limpieza de datos económicos descargados desde el Instituto Nacional de Estadística (INE), utilizando técnicas de Python para su manipulación y procesamiento eficiente.


In [ ]:
## **Índice**
1. Introducción
2. Funciones desarrolladas y lógica aplicada
3. Módulos utilizados
4. Resultados
5. Bibliografía

---

## **Introducción**

Este trabajo se enfoca en la manipulación y limpieza de un dataset económico descargado desde el sitio oficial del Instituto Nacional de Estadística (INE). El objetivo es procesar un archivo Excel con más de 30 filas y 100 datos, limpiando los datos faltantes o erróneos, transformando la estructura del dataset, y finalmente calculando estadísticas como media, varianza y moda.

La lógica aplicada a este trabajo se basa en la **Programación Orientada a Objetos (POO)** para encapsular los datos en una clase personalizada, permitiendo manipular las columnas del dataset como atributos de objetos.

Se presentarán las funciones clave desarrolladas, los módulos de Python utilizados, y los resultados obtenidos tras aplicar dichas funciones al dataset.

![Ejemplo de dataset limpio](https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Example_of_data_sheet.svg/2000px-Example_of_data_sheet.svg.png)


In [2]:
!pip install pandas requests openpyxl

In [2]:
import pandas as pd
import requests
from IPython.display import display

In [20]:
def descargar_datos(url, file_path):
    try:
        # Descargar el archivo y guardarlo localmente
        response = requests.get(url)
        response.raise_for_status()  # Verificar si la solicitud fue exitosa
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print("Archivo descargado correctamente.")
        # Leer el archivo Excel en un DataFrame
        df = pd.read_excel(file_path, engine='openpyxl')
        print("Datos cargados en el dataset.")
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error en la descarga del archivo: {e}")
    except Exception as e:
        print(f"Error al cargar los datos: {e}")

# Uso
url = "https://www.ine.es/jaxiT3/files/t/es/xlsx/62271.xlsx?nocab=1"
file_path = "dataset.xlsx"
df = descargar_datos(url, file_path)

Archivo descargado correctamente.
Datos cargados en el dataset.


C:\Users\CampusFP\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [22]:
def limpiar_datos(df):
    # Reemplazar valores NaN por 0
    df.fillna(0, inplace=True)
    # Eliminar columnas sin nombre
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    # Convertir valores no numéricos en columnas numéricas a 0 (opcional)
    for col in df.select_dtypes(include='number').columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    print("Datos limpiados correctamente.")
    return df

# Uso
df = limpiar_datos(df)
display(df.head())  # Mostrar las primeras filas del dataset limpio


Datos limpiados correctamente.


,Cuentas trimestrales no financieras de los sectores institucionales. Revisión 2024.
0,S11. Sociedades no financieras
1,0
2,Saldos contables
3,"Unidades: %, Millones Euros"
4,0


In [ ]:
# Parte Reflexiva: Limpieza de Datos

### Alternativa Investigada:
Utilicé una IA generativa para investigar diferentes métodos para la limpieza de datos en Python. Un método alternativo sería el uso de la función `dropna()`, que permite eliminar filas o columnas que contienen valores nulos (NaN).

### Reflexión:
He optado por utilizar el método `fillna()` en lugar de `dropna()` en mi ejercicio. La razón principal es que, en un dataset económico, cada fila puede representar datos importantes y eliminar una fila completa debido a valores faltantes podría resultar en la pérdida de información crucial. Al rellenar los valores faltantes con 0, se preservan todas las observaciones, lo que asegura que el análisis posterior incluya todos los datos disponibles.

### Mejora del Código:
Con esta elección, mi código es más robusto, ya que gestiona los valores faltantes sin eliminar filas. Este enfoque asegura una manipulación más eficiente del dataset y mantiene un manejo adecuado de excepciones. De esta forma, se asegura que el dataset esté listo para su análisis sin perder información relevante.


In [34]:
def guardar_datos(df, file_name):
    try:
        with open(file_name, 'w') as f:
            for row in df.values:
                f.write(' '.join(map(str, row)) + '\n')
        print(f"Datos guardados en {file_name}")
    except Exception as e:
        print(f"Error al guardar los datos: {e}")

# Uso
guardar_datos(df, 'lista.txt')


Datos guardados en lista.txt


In [26]:
def calcular_estadisticas(df):
    try:
        df_numerico = df.select_dtypes(include='number')
        moda = df_numerico.mode()
        estadisticas = {
            'media': df_numerico.mean(),
            'varianza': df_numerico.var(),
            'moda': moda.iloc[0] if not moda.empty else 'No hay moda'
        }
        print("Estadísticas calculadas correctamente.")
        return estadisticas
    except Exception as e:
        print(f"Error al calcular estadísticas: {e}")
        return {}

# Uso
estadisticas = calcular_estadisticas(df)
print(estadisticas)


Estadísticas calculadas correctamente.
{'media': Series([], dtype: float64), 'varianza': Series([], dtype: float64), 'moda': 'No hay moda'}


In [28]:
class Dataset:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __str__(self):
        return str(self.__dict__)

    def actualizar_atributo(self, atributo, valor):
        if atributo in self.__dict__:
            setattr(self, atributo, valor)
        else:
            print(f"Atributo {atributo} no encontrado.")

    def __eq__(self, otro):
        return self.__dict__.get('columna2', 0) == otro.__dict__.get('columna2', 0)

    def __add__(self, otro):
        return Dataset(**{k: v + otro.__dict__.get(k, 0) for k, v in self.__dict__.items()})

    def __sub__(self, otro):
        return Dataset(**{k: v - otro.__dict__.get(k, 0) for k, v in self.__dict__.items()})

# Uso
obj1 = Dataset(columna1=1, columna2=2)
obj2 = Dataset(columna1=3, columna2=4)

print(obj1 + obj2)
print(obj1 - obj2)


{'columna1': 4, 'columna2': 6}
{'columna1': -2, 'columna2': -2}


In [30]:
def mostrar_datos(file_path):
    try:
        df = pd.read_excel(file_path, engine='openpyxl')
        display(df)
        print("Datos cargados correctamente.")
        return df
    except Exception as e:
        print(f"Error al cargar los datos desde el archivo: {e}")
        return None

# Uso
df = mostrar_datos("dataset.xlsx")


C:\Users\CampusFP\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Cuentas trimestrales no financieras de los sectores institucionales. Revisión 2024.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,S11. Sociedades no financieras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Saldos contables,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Unidades: %, Millones Euros",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,Notas:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Datos cargados correctamente.


In [38]:
# Celdas de código con los resultados de la limpieza de datos, transformación y cálculos estadísticos

# Función para limpiar los datos
def limpiar_datos(df):
    df.fillna(0, inplace=True)  # Rellenar NaN con 0
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # Eliminar columnas sin nombre
    return df

# Descargar y cargar el dataset desde INE
import pandas as pd
url = "https://www.ine.es/jaxiT3/files/t/es/xlsx/62271.xlsx?nocab=1"
df = pd.read_excel(url, engine='openpyxl')

# Aplicar la función de limpieza
df_limpio = limpiar_datos(df)

# Cálculo de estadísticas
def calcular_estadisticas(df):
    df_numerico = df.select_dtypes(include='number')
    moda = df_numerico.mode()
    estadisticas = {
        'media': df_numerico.mean(),
        'varianza': df_numerico.var(),
        'moda': moda.iloc[0] if not moda.empty else 'No hay moda'
    }
    return estadisticas

estadisticas = calcular_estadisticas(df_limpio)

# Mostrar los resultados de las estadísticas
estadisticas


C:\Users\CampusFP\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


{'media': Series([], dtype: float64),
 'varianza': Series([], dtype: float64),
 'moda': 'No hay moda'}

In [ ]:
## **Webgrafía y Bibliografía**

1. Instituto Nacional de Estadística (INE). (2024). *Datos estadísticos económicos de España*. Recuperado de https://www.ine.es
2. McKinney, W. (2017). *Python for Data Analysis: Data Wrangling with Pandas, NumPy, and IPython*. O'Reilly Media.
3. Pedregosa, F., et al. (2011). *Scikit-learn: Machine Learning in Python*. Journal of Machine Learning Research, 12, 2825-2830.
4. Wes McKinney. (2010). *Data Structures for Statistical Computing in Python*. Proceedings of the 9th Python in Science Conference.
5. NumPy Developers. (2021). *NumPy Reference*. Disponible en https://numpy.org/doc/
